In [1]:
from or_gym.envs.classic_or.knapsack import OnlineKnapsackEnv

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [2]:
env=OnlineKnapsackEnv()
state=env._RESET()
print(f"State for OnlineKnapsackEnv {state}")

total_reward = 0
done = False

###################### Random sampling #####################################################

print("Starting Online Knapsack Simulation")
print("Initial state:", state)

# Run simulation until episode ends
while not done:
    # Choose an action randomly: 0 (reject) or 1 (accept)
    action = env.sample_action()
    print(action)
    # Take a step in the environment
    next_state, reward, done, info = env._STEP(action)
        
    total_reward += reward
    print(f"Action: {'Accept' if action == 1 else 'Reject'} | Reward: {reward} | Next state: {next_state} | Done: {done}")
    exit()
print(f"Episode finished. Total reward: {total_reward}")

State for OnlineKnapsackEnv {'state': array([  0, 146,   4,  19]), 'avail_actions': array([1, 1], dtype=uint8), 'action_mask': array([1, 1], dtype=uint8)}
Starting Online Knapsack Simulation
Initial state: {'state': array([  0, 146,   4,  19]), 'avail_actions': array([1, 1], dtype=uint8), 'action_mask': array([1, 1], dtype=uint8)}
1
Action: Accept | Reward: 19 | Next state: {'state': array([  4, 137,  99,  46]), 'avail_actions': array([1, 1], dtype=uint8), 'action_mask': array([1, 1], dtype=uint8)} | Done: False
0
Action: Reject | Reward: 0 | Next state: {'state': array([  4, 191,  14,  68]), 'avail_actions': array([1, 1], dtype=uint8), 'action_mask': array([1, 1], dtype=uint8)} | Done: False
1
Action: Accept | Reward: 68 | Next state: {'state': array([ 18, 101,  47,   3]), 'avail_actions': array([1, 1], dtype=uint8), 'action_mask': array([1, 1], dtype=uint8)} | Done: False
1
Action: Accept | Reward: 3 | Next state: {'state': array([ 65, 129,  96,  53]), 'avail_actions': array([1, 1], 